# **第3章：ベイズ統計の基礎とPyMC3**

ここではベイズ統計におけるモデル設計方法や計算方法に関する基礎事項を解説します．
また，PyMC3を使って設計したモデルから自動的に推論計算を行う方法に関しても解説します．
条件付き確率の確認

## **3.1 周辺分布と条件付き分布**
通常，ベイズ統計では単一の変数を扱うことはなく，複数の変数間の関係性をモデル化します．

2つの確率変数$x$，$y$をもつ確率分布$p(x, y)$を**同時分布（joint distribution）**と呼びます．ベイズ統計における最も重要な計算は次の2つになります．

**周辺分布（marginal distribution）の計算**

【連続】
$$
p(y) = \int p(x, y) {\rm d}x
$$

【離散】
$$
p(y) = \sum_x p(x, y)
$$

簡単に言うと，周辺分布$p(y)$とは，あらゆる$x$の影響をすべて考慮したうえで，$y$の分布だけを考えることです．

**条件付き分布（conditional distribution）の計算**
$$
p(x|y) = \frac{p(x, y)}{p(y)}
$$
簡単に言うと，周辺分布$p(x|y)$とは，ある特定の$y$の値が与えられた場合の$x$の分布を考えることです．

なお，$p(x, y)$における$x$と$y$は通常依存関係を持っています（$x$の出方によって$y$の出方が変わる，等）．
依存関係を持っていない場合は$p(x, y)=p(x)p(y)$と書くことができ，このとき$x$と$y$は**独立（independent）**であると言います．

ベイズ統計における解析とは，

**１．同時分布$p(x, y)$を設計し，**

**２．条件付き分布$p(x|y)$を計算する**

ことに他なりません．



## **3.2 選ばれた袋はどっち？**
簡単な確率計算の例を使って，ベイズ統計におけるモデル設計と推論計算の基本を確認しましょう．

#### **同時分布の設計（モデルの設計）**

![代替テキスト](https://drive.google.com/uc?id=1dvSWDu90dtPA4ATxdfbcbobM4RIDVCkT)

上図のように，赤玉3個，白玉1個が入った袋aと，赤玉1個，白玉3個が入った袋bがあります．
まず，50％の確率で袋a,bのいずれかを選びます．
次に，選んだ袋から１つだけ玉を取り出して，その色を確認します．
この作業を確率モデルで表現してみましょう．

選んだ袋を$x$とすれば，$x=a$および$x=b$となる確率は次のように書けます．
$$
p(x=a) = 1/2 \\
p(x=b) = 1/2
$$

また，それぞれの袋a,bで，赤玉が出る確率は玉の比率から$2/3$，$1/4$であることわかります．これらはそれぞれ袋が$x=a$だとわかったとき，および$x=b$だとわかったときの玉の条件付き確率です．
$$
p(y=赤|x=a) = 2/3 \\
p(y=赤|x=b) = 1/4
$$

当然，白玉が出る条件付き確率も簡単に計算できます．
$$
p(y=白|x=a) = 1/3 \\
p(y=白|x=b) = 3/4
$$

一般的に同時分布は（条件付き分布の定義から）$p(x, y) = p(y|x)p(x)$と書けるので，上記の確率の値をすべて使えば同時分布$p(x,y)$が設計できたことになります．

#### **（袋の）条件付き分布の計算**
さて，モデル$p(x,y)$を設計しただけでは何も有益な解析することができません．
モデルは，データ（あるいは情報）が新たに与えられた場合に，意味のある解析を行うことができます．
ここでは，先ほどの玉の取り出し作業を行い，「出た玉が赤玉だった場合」を考えてみることにします．
この場合，**もともと選ばれた袋は$a$,$b$どちらの可能性が高いでしょうか？**

これは，条件付き分布$p(x|y=赤)$を計算することで確認することができます．
簡単のため，袋が$x=a$となる確率を考えてみましょう．条件付き分布の定義から，
$$
p(x=a|y=赤) = \frac{p(x=a, y=赤)}{p(y=赤)}
$$
となります．分母$p(y=赤)$をまず計算します．周辺分布の定義から
$$
\begin{align}
p(y=赤)&= \sum_{x \in \{a, b\}} p(x, y=赤) \\
&= p(x=a, y=赤) + p(x=b, y=赤) \\
&= p(y=赤|x=a)p(x=a) + p(y=赤|x=b)p(x=b) \\
&= \frac{2}{3} \times \frac{1}{2} + \frac{1}{4} \times \frac{1}{2} \\
&= \frac{11}{24}
\end{align}
$$

となります．分子$p(x=a, y=赤)$も，
$$
\begin{align}
p(x=a, y=赤) &=p(y=赤|x=a)p(x=a) \\
&= \frac{2}{3} \times \frac{1}{2} \\
&= \frac{1}{3}
\end{align}
$$
なので，結局 
$$
\begin{align}
p(x=a|y=赤) &= \frac{1/3}{11/24} \\
&= \frac{8}{11}
\end{align}
$$
となります．すなわち「取り出した玉が赤玉だった場合，$a$の袋が選ばれた確率は$8/11 \approx 0.73$程度であることがわかります．選ばれた袋は$a$であった可能性の方が高いと言えます．

#### **推論，事前分布，事後分布**

このように，モデル$p(x,y)$に対してデータ$y$を与え，条件付き分布$p(x|y)$を調べることを**確率推論（probabilistic inference）**あるいは単に**推論（inference）**と呼びます．

![代替テキスト](https://drive.google.com/uc?id=149OZAWewXZ-K2nbb7LWW7afFmDiEDSJs)

また，この場合$p(x|y)$は「データ$y$を観測した後の$x$の分布」を表しているため，**事後分布（posterior distribution）**と呼ばれます．一方で$p(x)$はデータ観測前にモデル設計者により設定されているため，**事前分布（prior distribution）**と呼ばれます．



#### **3.2.1 やってみよう(10分)**
以下の条件のもと 「取り出した玉が赤玉だった場合」に、もともと「選ばれた袋が a である確率」を計算してください
 - 袋 a : 赤玉4個 白玉2個
 - 袋 b : 赤玉2個 白玉3個
 - 50%の確率で袋a,bいずれかを選ぶ

## **3.3 複数の観測データを解析する場合**
さきほどは袋から玉を取り出す事象を例にして，解析対象に対して確率モデルを構築し，周辺分布や条件付き分布やの計算によって，ある種の確率的な結論を導けることを示しました．
同じ考え方は，**観測データが複数存在するようなより一般的なケース**にも適用できます．

#### **球を複数回取り出す場合**
先ほどと同様に，まず，50％の確率で袋a，bのいずれかを選びます．
次に，**選んだ袋xから玉を1つ取り出して色yを確認し，xに戻すという試行を3回繰り返す**ことにします．

今回は玉を3回取り出すので，出た色の結果は$y_1$，$y_2$，$y_3$のように3つの変数で表すことにします．一方で袋の選択は1度だけなので，先ほどと同様$x$で表します．このモデル（同時分布）は次のように書くことができます．

$$
p(y_1, y_2, y_3, x) = p(x)p(y_1|x)p(y_2|x)p(y_3|x) 
$$

さて，取り出した3つの玉の記録が次のようなものになったとします．
$$
\{ y_1=赤, y_2=赤, y_3=白 \}
$$

このとき，選んだ袋がx=aであった確率はいくつになるでしょうか？
これは，先ほどと同じ手順で条件付き分布$p(x | y_1=赤, y_2=赤, y_3=白)$を求めることによって得られます．（詳細な解法は「ベイズ推論による機械学習入門」参照）

$$
\begin{align}
&p(x=a | y_1=赤, y_2=赤, y_3=白) \\
&= \frac{p(x=a)p(y_1|x=a)p(y_2|x=a)p(y_3|x=a) }{p(y_1=赤, y_2=赤, y_3=白)} \\
&\ldots（中略） \ldots \\
&= \frac{256}{337}
\end{align}
$$

ここでは，得られた玉の記録が$\{ y_1=赤, y_2=赤, y_3=白 \}$であった場合，選ばれた袋がaである確率は$256/337 \approx 0.76$であることがわかりました．



## **3.4 PyMC3を使ってみよう．**
- 赤玉白玉の例を使って，次のステップを解説しました．
  - 【確率モデルの設計】玉を取り出す手続きを同時分布で表現する
  - 【推論計算】観測データを与え，選ばれた袋の条件付き分布を算出する
- 統計モデリングにおける一般的な解析手順も全く同様です．ただし，現実の問題に役に立つようなモデルでは，上記の【推論計算】が非常に複雑になるため，手計算を行うことができません．
- ここでは，**PyMC3**というベイズ統計のPythonライブラリを使うことによって，**赤玉白玉問題における推論計算をコンピュータに行わせる**ことにします．

In [5]:
import numpy as np
import matplotlib.pyplot as plt

# ベイズ統計ライブラリ
import pymc3 as pm

# 計算時間計測用
import time

# 後ほどモデル構築のために利用（必須ではない）
import theano.tensor as tt

print('Running on PyMC3 v{}'.format(pm.__version__))

#### **１つの玉を観測した場合**

In [ ]:
# 【数値の対応表】
# 袋a：x=0, 袋b：x=1
# 赤玉：y=0，白玉:y=1
#このアルゴリズムは正確な確率が計算できるわけじゃない（あくまで近似値）。だけど複雑なモデルでも計算できる

# 観測データ（赤玉を１つ取り出した）
y = 0

# モデルの設計
with pm.Model() as model:
  # p(x) 2つある袋のうち、一つを選ぶ確率
  pm_x = pm.Categorical('pm_x', p = [1/2, 1/2]) #カテゴリカル分布、第一引数は名前、第二引数はパラメータ

  # p(y|x) 球が各袋から選ばれる確率。
  pm_y = pm.Categorical('pm_y', p = tt.stack([[2/3, 1/3], [1/4, 3/4]])[pm_x], observed=y) #observedは観測データと突き合わせをする部分。

  # MCMCのサンプルサイズ（後述）
  N_samples = 10000

  # 時間計測用
  start = time.time()

  # MCMCによる事後分布からのサンプリング（後述）
  trace = pm.sample(N_samples, chains=1)
  
  print('elapsed time: {:.9f} [sec]'.format(time.time() - start))



In [ ]:
# p(x=a|y=赤)の計算

# 先ほど手計算で得た厳密な解
print(8/11)

# PyMC3で近似的に計算した解
print(np.mean(trace['pm_x'] == 0))

# 絶対誤差
print(np.abs(8/11 - np.mean(trace['pm_x'] == 0)))


#### **PyMC3コードの解説**
ここでPyMC3で行ったことは，赤玉白玉問題を手計算で計算した場合と同じです．
実装の便宜上，ここでは次のようなバイナリを使った表記を行っています．

```
# 袋a ： x=0,  袋b ： x=1
# 赤玉： y=0， 白玉:  y=1
```

**【観測データ】**

上記の表記法を用いれば，「観測された球は赤玉だった」は次のように書けます．(観測データ)
```
y = 0
```

**【モデル設計】**

次のコードでは，モデルの設計（同時分布$p(x,y)=p(y|x)p(x)$の設計）を行っています．
2値なのでベルヌーイ分布も使えますが，ここでは次元数が2のカテゴリ分布を使っています（数学的にはまったく等価）．袋の選択$p(x)$は確率$1/2$ずつを割り当てています．玉の取り出し$p(y|x)$は条件付き分布になっています．tt.stack()で行列を作っていますが，pm_xで指定された行で玉の出る比率が指定されていることになります．
```
with pm.Model() as model_linear:
  # p(x)
  pm_x = pm.Categorical('pm_x', p = [1/2, 1/2])
  # p(y|x)
  pm_y = pm.Categorical('pm_y', p = tt.stack([[2/3, 1/3], [1/4, 3/4]])[pm_x], observed=y)
```

**【サンプリングアルゴリズム（MCMC）による推論計算】**

モデル設計が終わった後は，関数pm.sample()により**サンプリングアルゴリズム**を実行することによって自動的に推論計算を行います．
```
    N_samples = 10000
    trace = pm.sample(N_samples, chains=1, random_seed=1)
```
サンプリングアルゴリズムは事後分布$p(x|y)$からの具体的なサンプルをN_samples個だけ抽出し，traceに格納します．PyMC3では標準サンプリングアルゴリズムとして**MCMC（Markov chain Monte Carlo, マルコフ連鎖モンテカルロ法）**を利用しています．MCMCで得られる$x$のサンプルは，事後分布$p(x|y)$からの厳密なサンプルではなく，近似になっています．近似の精度はサンプルサイズN_samplesによって決まります．**N_samplesを大きくすることによって近似の精度が上がりますが，計算量が増えます**．

**【サンプルの取り出しと推定値の計算】**

traceには$x$のN_samples個の抽出されたサンプルが格納されています．

In [ ]:
print(trace['pm_x'])
plt.hist(trace['pm_x'], bins=2)


次の行でtraceに格納されたサンプリングの結果を使って，取り出された袋の確率を近似的に算出しています．ここでは袋がaだった確率（xが0）を計算しています．

```
# PyMC3で近似的に計算した解
print(np.mean(trace['pm_x'] == 0))
```

#### **3.4.1 やってみよう（5分）**
N_samplesを色々変えてみて再度モデルを定義してからMCMCを実行し，推定結果がどのように変わるか見てみましょう．
下記の表を埋める要領で実験してみてください．

|N_samples|計算時間（sec）|p(x=a｜y=赤)の推定値|厳密解8/11との絶対誤差|
|---|---|---|---|
|100|   |   |   |
|1000|   |   |   |
|10000|1.672|0.7243|0.002973|
|100000|   |   |   |


## **3.5 複数の玉を観測した場合**
同じようにして，玉を3回取り出したケースに関してもMCMCを使って近似計算してみましょう．

In [ ]:
# 【数値の対応表】
# 袋a：x=0, 袋b：x=1
# 赤玉：y=0，白玉:y=1

# 観測データ
# {y_1=赤, y_2=赤, y_3=白}
y = [0, 0, 1]

# モデル設計
with pm.Model() as model:
  #p(x)
  pm_x = pm.Categorical('pm_x', p = [1/2, 1/2])

  # p(y_1|x)p(y_2|x)p(y_3|x)
  # observedにリストを代入すると，自動的にブロードキャストされる．
  pm_y = pm.Categorical('pm_y', p = tt.stack([[2/3, 1/3], [1/4, 3/4]])[pm_x], observed=y)
  
  # なお，ブロードキャストを使わず，3回書いても良い．
  #pm_y1 = pm.Categorical('pm_y1', p = tt.stack([[2/3, 1/3], [1/4, 3/4]])[pm_x], observed=y[0])
  #pm_y2 = pm.Categorical('pm_y2', p = tt.stack([[2/3, 1/3], [1/4, 3/4]])[pm_x], observed=y[1])
  #pm_y3 = pm.Categorical('pm_y3', p = tt.stack([[2/3, 1/3], [1/4, 3/4]])[pm_x], observed=y[2])

  # MCMCのサンプルサイズ
  N_samples = 10000

  # MCMCによる事後分布からのサンプリング
  trace = pm.sample(N_samples, chains=1)

In [ ]:
# 先ほど手計算で得た厳密な解
print(256 / 337)

# PyMC3で近似的に計算した解
print(np.mean(trace['pm_x'] == 0))


#### **3.5.1 やってみよう（5分）**
データ数を増やすと推定結果はどのように変化するでしょうか？
例えばyを
$$
y = [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]
$$
などとしてみて，再度モデルを作成し，MCMCを実行してみましょう．

## **3.6 MCMC(マルコフ連鎖モンテカルロ法）は何をやっているのか？**

#### **モデル，データ，事後分布**
ベイズ統計の課題は，観測データの集合を${\bf Y}$，未観測のデータの集合を${\bf X}$とすると，次の手続きを実行することになります．
- 【確率モデルの設計】課題に合わせて$p({\bf Y}, {\bf X})$を作る．
- 【推論計算】条件付き分布$p({\bf X}|{\bf Y})$を計算し，${\bf X}$の分布を調べる．
特に，データが条件として与えられた後の分布$p({\bf X}|{\bf Y})$は**事後分布（posterior）**と呼びます．事後分布を精緻に計算することがベイズ統計の応用上で最も重要です．

#### **事後分布のサンプルによる近似**
赤玉白玉の例は非常に簡単なので，$p({\bf X}|{\bf Y})$は解析的に手計算で求めることができました（結果はベルヌーイ分布）．
しかし，条件付き分布$p({\bf X}|{\bf Y})$は多くの場合簡単には計算できません．
一般的に事後分布は
$$
p({\bf X}|{\bf Y}) = \frac{p({\bf X}, {\bf Y})}{p({\bf Y})}
$$
と書くことができます．この式の分子は，

$$
\begin{align}
&【{\bf X}が連続】p({\bf Y})=\int p({\bf Y}, {\bf X}) {\rm d}{\bf Y} \\
&【{\bf X}が離散】p({\bf Y})=\sum_{\bf X} p({\bf Y}, {\bf X})
\end{align}
$$

となりますが，この**積分または和の計算が非常に困難になるケースが多い**のです．
（積分の場合は解析計算不能，和の計算の場合は組み合わせ爆発，など）
したがって，積分や和の計算を厳密に行わなくても，事後分布$p({\bf X}|{\bf Y})$の性質を調べる方法が望まれます．

確率分布の性質を調べる方法は次のようなものがありました．

１． プロットして概形を描いてみる．

**２． 値をシミュレーション（=サンプリング）してみる．**

３．（平均や分散などの）量を計算してみる．

一般的な統計モデリングでは，高次元のパラメータを取り扱ったりするので，1の方法はあまり現実的ではないでしょう．
3の方法は，2の方法によって具体的なサンプル点が集まれば，そこからmeanやstd関数を使って計算できます．
**MCMCは，厳密に計算できない事後分布$p({\bf X}|{\bf Y})$から，その分布に従う値${\bf X}$を複数個シミュレート（サンプリング）してくれる方法です．**

MCMCの原理や代表的なアルゴリズムは下記の書籍が参考になります．
- パターン認識と機械学習（PRML）
  - https://www.maruzen-publishing.co.jp/item/b294551.html
- MLPシリーズ ベイズ深層学習
  - https://www.kspub.co.jp/book/detail/5168707.html
